# SVM
- 다목적 머신러닝 : 선형, 비선형 분류, 회귀, 이상치 탐색에 사용 가능
- 복잡한 분류문제, 작거나 중간 크기의 데이터셋에 적합

## 선형 SVM 분류
- 데이터를 분류하면서 가장 폭이 넓은 도로가 되는 결정 경계를 찾는 방법. =Large margin classification
- 도로 바깥쪽에 결정 경계에 영향을 미치는 샘플이 있는데 이 샘플을 support vector라고 한다.
- 스케일에 민감함. 

### 하드 마진 분류 / 소프트 마진 분류
- 하드 마진 분류 : 모든 샘플이 도로 바깥쪽에 올바르게 분류되어있는 분류.
    + 그러나 문제점이 있음. 데이터가 선형적으로 구분되어있어야 제대로 작동하며 이상치에 민감함.
- 소프트 마진 분류 : 하드 마진 분류는 현실적으로 어려운 부분이 있기 때문에 마진 안에 또는 결정 경계 반대쪽에 있는 마진 오류를 어느 정도 감수하고(하이퍼파라미터 C가 조절) 결정 경계를 만드는 것.
    +  C=하드하게 분류하는 정도. C를 낮추면 마진이 넓어지고 과대적합을 막을 수 있다.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2,3)]
y = (iris["target"]==2).astype(np.float64)

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")) 
    #또는 ("linear_svc",SVC(kernel="linear", C=1))로 쓸 수 있다.
])

svm_clf.fit(X,y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [2]:
svm_clf.predict([[5.5,1.7]])

array([1.])

## 비선형 SVM 분류

### 다항 특성 추가 + linear SVM

### 다항식 커널

### 유사도 특성

### 가우시안 RBF 커널

### 계산복잡도

## SVM 회귀

## SVM 이론